In [1]:
import numpy as np
import joblib
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D,Input, Flatten, Dense, LSTM, TimeDistributed, Reshape, Dropout
from tensorflow.keras.layers import ConvLSTM2D, Dense, Flatten, Reshape, Dropout  
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

# Load preprocessed data
X_train = np.load('data/X_train.npy')
y_train = np.load('data/y_train.npy')
X_val = np.load('data/X_val.npy')
y_val = np.load('data/y_val.npy')
#X_test = np.load('data/X_test.npy')
#y_test = np.load('data/y_test.npy')
scaler = joblib.load('data/scaler.pkl')

# Verify shapes
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_val shape:", X_val.shape)
print("y_val shape:", y_val.shape)
#print("X_test shape:", X_test.shape)
#print("y_test shape:", y_test.shape)

2025-03-31 12:32:00.293540: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-31 12:32:00.297110: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-31 12:32:00.390841: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-31 12:32:00.429627: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743417120.513075    8469 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743417120.53

X_train shape: (28027, 24, 11, 33, 8)
y_train shape: (28027, 11, 33, 8)
X_val shape: (6989, 24, 11, 33, 8)
y_val shape: (6989, 11, 33, 8)


In [ ]:
#ConvLSTM v4 

n_timesteps, n_lat, n_lon, n_vars = X_train.shape[1:]  # e.g., (24, 11, 33, 9)

# Define Sequential model
model = Sequential([
    # First ConvLSTM layer
    ConvLSTM2D(filters=32, kernel_size=(3, 3), padding='same',activation='relu',
               input_shape=(n_timesteps, n_lat, n_lon, n_vars),
               return_sequences=True),
    Dropout(0.2),
    # Second ConvLSTM layer
    ConvLSTM2D(filters=64, kernel_size=(3, 3), padding='same',activation='relu',
               return_sequences=False),
    Dropout(0.2),
    # Flatten to prepare for Dense layer
    Flatten(),
    # Dense layer to map to output shape
    Dense(n_lat * n_lon * n_vars, activation='linear'),
    
    # Reshape to match target shape (11, 33, 9)
    Reshape((n_lat, n_lon, n_vars))
])

# Compile model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])

# Train model 
history = model.fit(X_train, y_train, validation_data=(X_val, y_val),
                    epochs=20, batch_size=64, verbose=1)

# Save model
model.save('models/convlstm_model_v4_final.keras')
joblib.dump(scaler, 'scaler.pkl')
